# Tomato Leaf Disease Detection

## Importing the Libraries

In [3]:
import tensorflow as tf
import pandas as pd 
from tensorflow.keras.utils import to_categorical
import random
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Activation, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, array_to_img, img_to_array
from tensorflow.keras.models import Sequential
from glob import glob
import cv2 as cv
camera = cv.VideoCapture(0)

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing the Dataset

In [18]:
# Define Constants by re-sizing all the images
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Datasets/train'
test_path = '/content/drive/MyDrive/Datasets/test'

#### Inception V3

In [19]:
# Import the InceptionV3 model and here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [6]:
# We don't need to train existing weights
for layer in inception.layers:
    layer.trainable = False

In [21]:
# Folders in the Training Set
folders = glob('/content/drive/MyDrive/Datasets/train*')
folders

['/content/drive/MyDrive/Datasets/train']

In [23]:
# Model layers -> can add more if required
x = Flatten()(inception.output)
prediction = Dense(len(folders), activation='softmax')(x)

In [24]:
# Create a model object
model = Model(inputs=inception.input, outputs=prediction)

# View the structure of the model
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_94 (BatchN  (None, 111, 111, 32  96         ['conv2d_94[0][0]']              
 ormalization)                  )                                                           

In [25]:
# Defining the cost and model optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [26]:
# Using the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                                rotation_range=30,
                                width_shift_range=0.20,
                                height_shift_range=0.20,
                                rescale = 1./255,
                                shear_range = 0.20,
                                zoom_range = 0.20,
                                horizontal_flip = True,
                                vertical_flip = True,
                                fill_mode = 'nearest')
test_datagen = ImageDataGenerator(rescale = 1./255)

In [28]:
# Training Generator
train_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/test',
                                               target_size = (224, 224),
                                               batch_size = 32,
                                               class_mode = 'categorical')

Found 4970 images belonging to 10 classes.


In [29]:
# Testing Generator
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/train',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 4970 images belonging to 10 classes.


In [ ]:
# fit the model, it will take some time to execute
r = model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)
)

## Visualize the model training by plotting Loss Function and Accuracy

In [ ]:
# Plot the Loss and Accuracy
# Loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# Accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [34]:
# Saving the model as a h5 file

from tensorflow.keras.models import load_model
model.save('model.h5')

In [ ]:
y_pred = model.predict(test_set)
y_pred

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([1, 9, 1, ..., 5, 7, 3])